## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.ones_like(img)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function which might be used as a starting point once we want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


In [4]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

## Build a Lane Finding Pipeline



In [5]:
# Calculate the area of the triangle
def area(x1, y1, x2, y2, x3, y3):
   return (abs((x1*(y2-y3) + x2*(y3-y1)+ x3*(y1-y2))/2.0))

# Check if the point(x,y) is inside the triangle
def isInside(x1, y1, x2, y2, x3, y3, x, y):   
   # Calculate area of triangle ABC
   A = area (x1, y1, x2, y2, x3, y3)
 
   # Calculate area of triangle PBC */  
   A1 = area (x, y, x2, y2, x3, y3)
 
   # Calculate area of triangle PAC */  
   A2 = area (x1, y1, x, y, x3, y3)
 
   # Calculate area of triangle PAB */   
   A3 = area (x1, y1, x2, y2, x, y)
   
   # Check if sum of A1, A2 and A3 is same as A */
   return (A == A1 + A2 + A3)


# TODO: Build your pipeline that will draw lane lines on the test_images_output
# then save them to the test_images directory.
def edgeDetection(image):
    # grayscale the image
    img_gray = grayscale(image)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    img_blur_gray = gaussian_blur(img_gray, kernel_size)

    # Define threshold parameters for Canny and apply
    low_threshold = 50
    high_threshold = 100
    img_edges = canny(img_blur_gray, low_threshold, high_threshold)

    # Define the Hough transform parameters
    rho = 1
    theta = np.pi/180
    threshold = 1
    min_line_length = 50
    max_line_gap = 2

    # Run Hough on edge detected image
    lines = cv2.HoughLinesP(img_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on the image
    for line in lines:
        for x1,y1,x2,y2 in line:
            inside1 = isInside(150,550,500,250,900,550, x1, y1)
            inside2 = isInside(150,550,500,250,900,550, x2, y2)
            if (inside1 and inside2):
                cv2.line(image,(x1,y1),(x2,y2),(255,0,0),10)

    
    # Can be further improved with a triangle for the region of interest
#     triangle = np.array([ [150,550], [500,250], [900,550] ], np.int32)
#     masked_image = region_of_interest(combo, [triangle])

    return image

def applyEdgeDetection(dir):
    for imagename in os.listdir(dir):
        # reading in an image
        image = mpimg.imread("test_images/"+imagename)

        # printing out some stats
        #print('This image is:', type(image), 'with dimensions:', image.shape)
        out_image = edgeDetection(image)
        
        # saving the image to out_images directory
        print('saving image ...')
        mpimg.imsave("test_images_output/out_"+imagename, out_image)
        
#applyEdgeDetection("test_images/")

saving image ...
saving image ...
saving image ...
saving image ...
saving image ...
saving image ...


In [28]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [29]:
def process_image(image):
    result = edgeDetection(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [30]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:03<00:00, 70.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.99 s, sys: 702 ms, total: 3.7 s
Wall time: 3.54 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [31]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [32]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:10<00:00, 67.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 9.71 s, sys: 2.04 s, total: 11.8 s
Wall time: 10.6 s


In [33]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))